# Train CryptoBert for NER

In [1]:
import os

os.getcwd()

'E:\\Crypto\\Annotations\\NER'

In [2]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer('vocab.txt')

tokens = ['[UNK]', '[CLS]', '[SEP]', '[PAD]']

special_tokens = [tokenizer.token_to_id(i) for i in tokens]
special_tokens

[1, 2, 3, 0]

In [3]:
from prodigy.components.loaders import JSONL

data = JSONL('annotations.jsonl')

In [4]:
import pandas as pd

df = pd.DataFrame()

row = {'input_ids': '', 'token_type_ids': '', 'attention_mask': '', 'labels': ''}

for line in data:

    if line['answer'] == 'accept':

        try:
            row['input_ids'] = [token['tokenizer_id'] for token in line['tokens']]
            row['token_type_ids'] = [token['type_ids'] for token in line['tokens']]
            row['attention_mask'] = [token['attention_mask'] for token in line['tokens']]
            row['labels'] = [-100 if i['tokenizer_id'] in special_tokens else 0 for i in line['tokens']]
        except:
            pass

        try:
            for label in line['spans']:
                start = int(label['token_start'])
                end = int(label['token_end'])

                row['labels'][start] = 1
                for i in range(start, end):
                    row['labels'][i+1] = 2

        except:
            pass

    df = df.append(row, ignore_index=True)

In [5]:
assert len(df['input_ids']) == len(df['token_type_ids'])
assert len(df['input_ids']) == len(df['attention_mask'])
assert len(df['input_ids']) == len(df['labels'])

In [6]:
df

,input_ids,token_type_ids,attention_mask,labels
0,"[2, 1436, 2105, 2924, 1407, 1436, 35, 7975, 13...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 1, 0, 0, 0, 1, 0, 0, 0, 0, -100]"
1,"[2, 51, 11, 2489, 4611, 1436, 2174, 1436, 2255...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"[2, 1436, 1489, 2511, 15679, 12937, 1379, 2987...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,"[2, 1814, 1713, 2949, 1381, 1370, 1956, 1385, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,"[2, 1436, 1529, 5828, 1370, 10681, 1555, 18, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
95,"[2, 2005, 1683, 1467, 1940, 1383, 2278, 1410, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
96,"[2, 1410, 283, 61, 1837, 1675, 1530, 1968, 181...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
97,"[2, 51, 1692, 1538, 1411, 1436, 16, 1599, 1483...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,"[2, 1380, 3283, 1849, 1998, 3513, 1380, 2005, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


## Datasets

In [7]:
import datasets

features = datasets.Features(
    {
        'input_ids': datasets.Sequence(datasets.Value("int32")),
        'token_type_ids': datasets.Sequence(datasets.Value("int32")),
        'attention_mask': datasets.Sequence(datasets.Value("int32")),
        'labels': datasets.Sequence(datasets.ClassLabel(num_classes=3, names=['0', 'B-COIN', 'I-COIN'])),
    }
)

In [8]:
from datasets import Dataset

training = df.iloc[:80,:]
test = df.iloc[80:,:]

dataset_train = Dataset.from_pandas(training, features=features)
dataset_test = Dataset.from_pandas(test, features=features)

In [9]:
dataset_train.features['labels']

Sequence(feature=ClassLabel(num_classes=3, names=['0', 'B-COIN', 'I-COIN'], names_file=None, id=None), length=-1, id=None)

## BERT - NER Finetuning

In [10]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('./CryptoBERT', padding=True, truncation=True)

In [11]:
for token, label in zip(tokenizer.convert_ids_to_tokens(dataset_train["input_ids"][20]),dataset_train["labels"][20]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
need____________________________________ 0
fiat____________________________________ 0
but_____________________________________ 0
don_____________________________________ 0
'_______________________________________ 0
t_______________________________________ 0
want____________________________________ 0
to______________________________________ 0
sell____________________________________ 0
my______________________________________ 0
bitcoin_________________________________ 1
at______________________________________ 0
any_____________________________________ 0
cost____________________________________ 0
is______________________________________ 0
there___________________________________ 0
a_______________________________________ 0
way_____________________________________ 0
to______________________________________ 0
safely__________________________________ 0
get_____________________________________ 0
loans___________________________________ 0
on______

In [12]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('./CryptoBERT', num_labels=3)

Some weights of the model checkpoint at ./model were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./model and are newly initialized: ['classifier.w

In [19]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    "CryptoBERT-NER",
    no_cuda=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [15]:
import datasets
import numpy as np

metric = datasets.load_metric("seqeval")

label_list = ['0', 'B-COIN', 'I-COIN']

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions,
                   references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [16]:
import torch
torch.cuda.is_available()

True

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

***** Running training *****
  Num examples = 80
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 15


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 20
  Batch size = 16
E:\Crypto\winvenv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=15, training_loss=0.13428414662679036, metrics={'train_runtime': 138.726, 'train_samples_per_second': 1.73, 'train_steps_per_second': 0.108, 'total_flos': 19083001292064.0, 'train_loss': 0.13428414662679036, 'epoch': 3.0})

In [21]:
model.save_pretrained("CryptoBERT-NER")
tokenizer.save_pretrained("tokenizer")

Configuration saved in nermodel\config.json
Model weights saved in nermodel\pytorch_model.bin
tokenizer config file saved in tokenizer\tokenizer_config.json
Special tokens file saved in tokenizer\special_tokens_map.json


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

## Test

In [22]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
import json
config = json.load(open("CryptoBERT-NER/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("CryptoBERT-NER/config.json","w"))

In [23]:
from transformers import pipeline

model = AutoModelForTokenClassification.from_pretrained("CryptoBERT-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

loading configuration file nermodel\config.json
Model config BertConfig {
  "_name_or_path": "nermodel",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "B-COIN",
    "2": "I-COIN"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": "0",
    "B-COIN": "1",
    "I-COIN": "2"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file nermodel\pytorch_model.bin
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of

[{'entity': '0', 'score': 0.99573696, 'index': 1, 'word': 'i', 'start': 0, 'end': 1}, {'entity': '0', 'score': 0.9975593, 'index': 2, 'word': 'think', 'start': 2, 'end': 7}, {'entity': 'B-COIN', 'score': 0.98610383, 'index': 3, 'word': 'bitcoin', 'start': 8, 'end': 15}, {'entity': '0', 'score': 0.99839956, 'index': 4, 'word': 'is', 'start': 16, 'end': 18}, {'entity': '0', 'score': 0.9960913, 'index': 5, 'word': 'a', 'start': 19, 'end': 20}, {'entity': '0', 'score': 0.997276, 'index': 6, 'word': 'good', 'start': 21, 'end': 25}, {'entity': '0', 'score': 0.9943117, 'index': 7, 'word': 'investment', 'start': 26, 'end': 36}]


In [24]:
example = "I think bitcoin is a good investment. bitcoin can save the world"
ner_results = nlp(example)
print(ner_results)

[{'entity': '0', 'score': 0.9968875, 'index': 1, 'word': 'i', 'start': 0, 'end': 1}, {'entity': '0', 'score': 0.9980868, 'index': 2, 'word': 'think', 'start': 2, 'end': 7}, {'entity': 'B-COIN', 'score': 0.9888265, 'index': 3, 'word': 'bitcoin', 'start': 8, 'end': 15}, {'entity': '0', 'score': 0.99873, 'index': 4, 'word': 'is', 'start': 16, 'end': 18}, {'entity': '0', 'score': 0.99634784, 'index': 5, 'word': 'a', 'start': 19, 'end': 20}, {'entity': '0', 'score': 0.9978637, 'index': 6, 'word': 'good', 'start': 21, 'end': 25}, {'entity': '0', 'score': 0.9955788, 'index': 7, 'word': 'investment', 'start': 26, 'end': 36}, {'entity': '0', 'score': 0.9977976, 'index': 8, 'word': '.', 'start': 36, 'end': 37}, {'entity': 'B-COIN', 'score': 0.9850418, 'index': 9, 'word': 'bitcoin', 'start': 38, 'end': 45}, {'entity': '0', 'score': 0.99808645, 'index': 10, 'word': 'can', 'start': 46, 'end': 49}, {'entity': '0', 'score': 0.9990214, 'index': 11, 'word': 'save', 'start': 50, 'end': 54}, {'entity': '

In [25]:
example = "bitcoin bitcoin. bitcoin can save the world"
ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-COIN', 'score': 0.9825546, 'index': 1, 'word': 'bitcoin', 'start': 0, 'end': 7}, {'entity': 'B-COIN', 'score': 0.98775333, 'index': 2, 'word': 'bitcoin', 'start': 8, 'end': 15}, {'entity': '0', 'score': 0.99839133, 'index': 3, 'word': '.', 'start': 15, 'end': 16}, {'entity': 'B-COIN', 'score': 0.98801965, 'index': 4, 'word': 'bitcoin', 'start': 17, 'end': 24}, {'entity': '0', 'score': 0.9980869, 'index': 5, 'word': 'can', 'start': 25, 'end': 28}, {'entity': '0', 'score': 0.99900657, 'index': 6, 'word': 'save', 'start': 29, 'end': 33}, {'entity': '0', 'score': 0.99587846, 'index': 7, 'word': 'the', 'start': 34, 'end': 37}, {'entity': '0', 'score': 0.9953231, 'index': 8, 'word': 'world', 'start': 38, 'end': 43}]
